## Intro to Deep Learning with PyTorch
### The XOR Problem

Updated 08/21/2024 G. Chism, U of A InfoSci + DataLab

**Install required libraries**

For this case we will import _PyTorch_, _sklearn_, _pandas_, and _numpy_.

**To execute code Notebook cells:** Press _SHIFT+ENTER_

In [ ]:
#!pip install -q torch
#!pip install -q scikit-learn
#!pip install -q pandas
#!pip install -q numpy

It's best practice to have all of the libraries loaded at the top of the page

In [ ]:
# Import specific classes from PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Import preprocessing from Scikit-Learn
from sklearn.preprocessing import StandardScaler

# Import pandas and numpy
import pandas as pd
import numpy as np